In [4]:
import os
import numpy as np
import struct
import time
from pyopenms import MSExperiment, MSSpectrum, Peak1D, FileHandler, SpectrumAlignment

def convert_tof_to_mzml(input_filename, output_folder):
    # Проверка и создание выходной директории
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Определение имени выходного файла на основе входного и папки назначения
    output_filename = os.path.join(output_folder, os.path.basename(input_filename).replace('.tof', '.mzML'))

    # Открытие входного файла в бинарном режиме
    with open(input_filename, 'rb') as fid:
        # Переход к позиции, где хранится количество спектров и количество точек на спектр
        fid.seek(278)
        numMS = struct.unpack('I', fid.read(4))[0]  # Чтение количества спектров
        numPn = struct.unpack('I', fid.read(4))[0]  # Чтение количества точек на спектр

        # Переход к позиции, где хранятся калибровочные коэффициенты
        fid.seek(286)
        AA = struct.unpack('d', fid.read(8))[0]  # Чтение коэффициента AA
        BB = struct.unpack('d', fid.read(8))[0]  # Чтение коэффициента BB
        
        # Вычисление шкалы m/z с использованием калибровочных коэффициентов
        zz_read = AA * ((np.arange(numPn)) + BB) ** 2

        # Переход к позиции, где начинаются данные спектров
        fid.seek(512)
        MS_1 = np.zeros((numMS, numPn), dtype=int)  # Инициализация массива для хранения данных всех спектров

        # Определение значений для типов данных (константы)
        fi_t1 = 0x4000
        fi_t2 = 0x8000
        fi_t3 = 0xc000
        fi_3f = 0x3fff

        # Инициализация объекта MSExperiment для хранения всех спектров
        experiment = MSExperiment()

        # Обработка каждого спектра
        for ii in range(numMS):
            start_time = time.time()  # Время начала обработки спектра
            arr = np.zeros(numPn, dtype=int)  # Инициализация массива для хранения данных одного спектра
            i1, i0, nB = 1, 1, 1  # Инициализация индексов

            # Чтение пакетов данных спектра
            npPack = struct.unpack('I', fid.read(4))[0]
            Buf_pack = np.frombuffer(fid.read(npPack * 2), dtype=np.uint16)
            nbArrP = struct.unpack('I', fid.read(4))[0]
            Buf_MS = np.frombuffer(fid.read(nbArrP), dtype=np.uint8)

            # Обработка каждого пакета данных
            for p in range(npPack):
                fi_type = Buf_pack[p]  # Тип данных пакета
                type = fi_type & fi_t3  # Маска типа данных
                c = fi_type & fi_3f  # Маска значений данных

                i1 = i0 + c  # Обновление индекса

                # Обработка данных в зависимости от типа
                if type == fi_t1:
                    arr[i0-1:i1-1] = Buf_MS[nB-1:nB+i1-i0-1]
                    nB += i1 - i0
                elif type == fi_t2:
                    idx_range = np.arange(nB-1, nB+2*(i1-i0-1), 2)
                    if len(idx_range) > len(arr[i0-1:i1-1]):
                        idx_range = idx_range[:len(arr[i0-1:i1-1])]
                    arr[i0-1:i1-1] = Buf_MS[idx_range] + 256*Buf_MS[idx_range + 1]
                    nB += 2 * (i1 - i0)
                elif type == fi_t3:
                    idx_range = np.arange(nB-1, nB+4*(i1-i0-1), 4)
                    if len(idx_range) > len(arr[i0-1:i1-1]):
                        idx_range = idx_range[:len(arr[i0-1:i1-1])]
                    arr[i0-1:i1-1] = Buf_MS[idx_range] + 256*Buf_MS[idx_range + 1] + \
                                     65536*Buf_MS[idx_range + 2] + 16777216*Buf_MS[idx_range + 3]
                    nB += 4 * (i1 - i0)

                i0 = i1  # Обновление начального индекса

            MS_1[ii, :] = arr  # Сохранение данных спектра в общий массив
            print(f'Spectrum {ii + 1}/{numMS} processed.')  # Уведомление о завершении обработки спектра

            # Создание объекта MSSpectrum и заполнение его данными
            spectrum = MSSpectrum()
            spectrum.setMSLevel(1)  # Установка уровня MS
            spectrum.set_peaks([zz_read, arr])

            # Добавление метаданных
            scan_time = float(ii)   # Время сканирования соответствует индексу спектра (в минутах)
            total_ion_current = float(np.sum(arr))  # Total Ion Current
            base_peak_intensity = float(np.max(arr))  # Интенсивность базового пика
            base_peak_mz = float(zz_read[np.argmax(arr)])  # Base Peak m/z

            spectrum.setRT(scan_time)
            spectrum.setMetaValue("total ion current", total_ion_current)
            spectrum.setMetaValue("base peak intensity", base_peak_intensity)
            spectrum.setMetaValue("base peak m/z", base_peak_mz)

            # Добавление спектра в эксперимент
            experiment.addSpectrum(spectrum)

    # Выровнять спектры с использованием SpectrumAlignment
    aligner = SpectrumAlignment()
    params = aligner.getParameters()
    params.setValue("tolerance", 0.1)  # Установка толерантности для выравнивания
    params.setValue("is_relative_tolerance", "true")
    aligner.setParameters(params)

    for i in range(1, experiment.getNrSpectra()):
        spectrum1 = experiment.getSpectrum(i - 1)
        spectrum2 = experiment.getSpectrum(i)
        alignment = []
        aligner.getSpectrumAlignment(alignment, spectrum1, spectrum2)
        # Применение выравнивания к спектрам (это может включать изменение m/z значений или интенсивностей)

    # Сохранение эксперимента в файл mzML
    try:
        FileHandler().storeExperiment(output_filename, experiment)
        print(f'{output_filename} "сгенерирован"')
    except Exception as e:
        print(f'Ошибка при сохранении файла: {e}')

# Пример использования функции
convert_tof_to_mzml('air_chudo_1_1.tof', './path')


Spectrum 1/453 processed.
Spectrum 2/453 processed.
Spectrum 3/453 processed.
Spectrum 4/453 processed.
Spectrum 5/453 processed.
Spectrum 6/453 processed.
Spectrum 7/453 processed.
Spectrum 8/453 processed.
Spectrum 9/453 processed.
Spectrum 10/453 processed.
Spectrum 11/453 processed.
Spectrum 12/453 processed.
Spectrum 13/453 processed.
Spectrum 14/453 processed.
Spectrum 15/453 processed.
Spectrum 16/453 processed.
Spectrum 17/453 processed.
Spectrum 18/453 processed.
Spectrum 19/453 processed.
Spectrum 20/453 processed.
Spectrum 21/453 processed.
Spectrum 22/453 processed.
Spectrum 23/453 processed.
Spectrum 24/453 processed.
Spectrum 25/453 processed.
Spectrum 26/453 processed.
Spectrum 27/453 processed.
Spectrum 28/453 processed.
Spectrum 29/453 processed.
Spectrum 30/453 processed.
Spectrum 31/453 processed.
Spectrum 32/453 processed.
Spectrum 33/453 processed.
Spectrum 34/453 processed.
Spectrum 35/453 processed.
Spectrum 36/453 processed.
Spectrum 37/453 processed.
Spectrum 3